In [1]:
import os
import sys
import numpy as np
import pandas as pd

import math

sys.path.append('../')
from utils import *
from save_data import *

from IPython.display import display, HTML
display(HTML("<style>.jp-OutputArea-output {display:flex}</style>"))

2024-02-07 12:34:42.981309: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-07 12:34:42.987899: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.1/lib64:/usr/local/cuda-10.1/lib64:/usr/local/cuda-10.1/lib64:/usr/local/cuda-10.1/lib64:/usr/local/cuda-10.1/extras/CUPTI/lib64:/usr/local/cuda-10.1/extras/CUPTI/lib64:/usr/local/cuda-10.1/extras/CUPTI/lib64:/usr/local/cuda-10.1/extras/CUPTI/lib64
2024-02-07 12:34:42.987910: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [12]:
df = pd.read_csv('../expt_phys_data/full_dataset_hilo.csv')
df = df[df.train_val_test == 0]#.reset_index(inplace=True)
df.reset_index(inplace=True,drop=True)
df

,id,smiles,iupac name,expt,tip3p,cha,zap9,mbondi,asc,igb5,null,train,train_val_test
0,mobley_1017962,CCCCCC(=O)OC,methyl hexanoate,-2.49,-3.299,-3.726220,-3.004340,-4.006280,-3.886,-5.059,0,False,0
1,mobley_1019269,CCCCO,butan-1-ol,-4.72,-3.232,-3.135990,-3.222270,-5.188390,-3.080,-4.198,0,False,0
2,mobley_1034539,c1cc(c(cc1c2cc(c(c(c2Cl)Cl)Cl)Cl)Cl)Cl,"1,2,3,4-tetrachloro-5-(3,4-dichlorophenyl)benzene",-3.04,-1.083,-2.344750,-2.113720,-0.459959,0.899,-1.348,0,False,0
3,mobley_1036761,C1CCC(CC1)N,cyclohexanamine,-4.59,-3.953,-1.755250,-0.447715,-0.027103,0.240,-2.215,0,True,0
4,mobley_1046331,c1ccc(cc1)OC=O,phenyl formate,-3.82,-5.442,-5.706530,-6.577580,-7.407990,-8.342,-10.084,0,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
557,mobley_9913368,C(=C/Cl)\Cl,"(E)-1,2-dichloroethylene",-0.78,1.024,-0.252349,-0.595880,-0.520564,-0.090,-1.126,0,True,0
558,mobley_9942801,CCc1ccc(cc1)C,1-ethyl-4-methyl-benzene,-0.95,-0.575,-1.201310,-0.421314,-0.758853,0.059,-1.641,0,True,0
559,mobley_994483,CCBr,bromoethane,-0.74,0.487,-0.821768,-0.062573,-1.089090,-0.095,-0.818,0,True,0
560,mobley_9974966,CC(C)SC(C)C,2-isopropylsulfanylpropane,-1.21,0.140,-1.111810,0.499588,-0.765837,0.032,-1.358,0,True,0


In [25]:
expt = df.expt.to_numpy()
train_index = list(psuedoScramble(expt, bins=int(len(expt)/10)))
val_index = []
for i in range(len(train_index)//7):
    k = i*6
    val_index.append(train_index.pop(k))
phys = ['tip3p','cha','zap9','mbondi','asc','igb5','null']
for p in phys:
    print(p,rmsd(df.expt.iloc[train_index],df[p].iloc[train_index]),rmsd(df.expt.iloc[val_index],df[p].iloc[val_index]))
    

tip3p 1.3680340636000716 1.3006224855814232
cha 1.490738009322944 1.2730290199171623
zap9 1.4968835946849661 1.4388644470341063
mbondi 2.03192711422197 2.015027460973036
asc 2.3127271802729874 2.319335031318244
igb5 2.625751961452288 2.42290085950705
null 3.8204911048394914 3.8515863614879517


In [27]:
df.loc[df.index[val_index],'train_val_test'] = 1
df.train_val_test.value_counts()

0    482
1     80
Name: train_val_test, dtype: int64

In [28]:
df_full = pd.read_csv('../expt_phys_data/full_dataset_hilo.csv')
for i in range(len(df_full)):
    for j in range(len(df)):
        if(df.at[j,'id'] == df_full.at[i,'id']):
            df_full.at[i,'train_val_test'] = df.at[j,'train_val_test']
df_full.train_val_test.value_counts()

0    482
1     80
2     80
Name: train_val_test, dtype: int64

In [29]:
phys = ['tip3p','cha','zap9','mbondi','asc','igb5','null']
for p in phys:
    print(p)
    for i in range(3):
        temp = df_full[df_full.train_val_test == i]
        print(rmsd(temp.expt,temp[p]))
  

tip3p
1.3680340636000714
1.3006224855814232
2.4702423742418476
cha
1.490738009322944
1.2730290199171623
2.955904469299963
zap9
1.496883594684966
1.4388644470341063
2.6014989016963854
mbondi
2.03192711422197
2.015027460973036
3.434827651037421
asc
2.3127271802729874
2.319335031318244
3.598451528088158
igb5
2.625751961452288
2.42290085950705
4.1872824585284425
null
3.8204911048394914
3.8515863614879517
11.485587327167906


In [30]:
df_full.to_csv('../expt_phys_data/full_dataset_hilo_with_val.csv',index=False)